In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sentence_transformers
!pip install flask_ngrok
!pip install pyngrok
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=c36a2dd07526b63113922d2d73ffcc149f01dfb5c839601ca53361a45c1d9779
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd

In [3]:
import pymongo
import pandas as pd
import torch
import json 
db_url = 'mongodb://ara-prod:LDBvT8AQvOdptEbDX2GU8sAL5ogUzDqpdvCHPQJgfSMmfLPqhWoLTXvuY8DTE0PPMuCLwOMgyGczACDbUJztYw==@ara-prod.mongo.cosmos.azure.com:10255/?ssl=true&retrywrites=false&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@ara-prod@'
DB_NAME = 'ARA_DB'
FEEDBACK_COLLECTION = 'FEEDBACK'
client = pymongo.MongoClient(db_url)
db = client[DB_NAME]
feedback_db = db[FEEDBACK_COLLECTION]
industries = ['ms_basic_materials', 'ms_communication_services', 'ms_consumer_cyclical', 'ms_consumer_defensive',
          'ms_energy', 'ms_financial_services', 'ms_healthcare', 'ms_industrials', 'ms_real_estate', 'ms_technology', 'ms_utilities']

In [4]:
# import pandas as pd
# import torch
# import json 

# PATH = '/content/drive/My Drive/2023 Spring/DataScinece Project/article_source/'
# industries = ['ms_basic_materials', 'ms_communication_services', 'ms_consumer_cyclical', 'ms_consumer_defensive', 
#               'ms_energy', 'ms_financial_services', 'ms_healthcare', 'ms_industrials', 'ms_real_estate', 'ms_technology', 'ms_utilities']

# for tmp in industries:
#   with open(PATH+tmp+'_news.json', 'r') as file:
#       globals()[tmp] = json.load(file)

In [5]:
import io
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok
import requests
!ngrok authtoken 2KRO36NhveZmYSb6Y0UShMHkMBK_3NDDN1wmF15haufztEEo4

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
import time
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from sentence_transformers import util

device = torch.device('cuda')

dpr_ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
dpr_ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base").to(device)
dpr_query_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-multiset-base")
dpr_query_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-multiset-base").to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


In [7]:
def select_candidates(query:str, sector:list, num=5)->list:
  contexts = []
  for i in range(len(sector)):
    contexts.append(sector[i]['content'])

  with torch.no_grad():
    dpr_query_input = dpr_query_tokenizer(query, return_tensors='pt')['input_ids']
    dpr_query_emb = dpr_query_encoder(dpr_query_input.to(device)).pooler_output
    dpr_ctx_input = dpr_ctx_tokenizer(contexts, padding=True, truncation=True, return_tensors='pt')
    dpr_ctx_emb = dpr_ctx_encoder(**dpr_ctx_input.to(device)).pooler_output

    dpr_query_input.to('cpu')
    dpr_ctx_input.to('cpu')
    del dpr_query_input
    del dpr_ctx_input
    torch.cuda.empty_cache()

    cos_scores = util.pytorch_cos_sim(dpr_query_emb,  dpr_ctx_emb)[0]
    top_results = torch.topk(cos_scores, k=num)

    top_k = top_results.indices.to('cpu')
    top_k = top_k.numpy()

    L = []
    urls = []
    for i in top_k:
      L.append(sector[i]['content'])
      urls.append(sector[i]['url'])

    return (L, urls)

In [8]:
# query = "Why did the Norwegian sovereign wealth fund oppose the election of its board chairman?"

# select_candidates(query, ms_energy)
# tmp = select_candidates(query, ms_energy)

In [9]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, TFPegasusForConditionalGeneration

# Let's load the model and the tokenizer 
model_name = "human-centered-summarization/financial-summarization-pegasus"
summary_tokenizer = PegasusTokenizer.from_pretrained(model_name)
summary_model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device) # If you want to use the Tensorflow model 
summary_model.eval()                                                   # just replace with TFPegasusForConditionalGeneration

def summarize(article_list:list):
  L = []
  for i in range(len(article_list)):
    original_article = article_list[i]
    with torch.no_grad():
      input_ids = summary_tokenizer(original_article, return_tensors="pt",truncation=True).input_ids.to(device)
      output = summary_model.generate(
        input_ids, 
        max_length=64, 
        num_beams=5, 
        early_stopping=True)
      result = summary_tokenizer.decode(output[0], skip_special_tokens=True)

      input_ids.to('cpu')
      del input_ids
      torch.cuda.empty_cache()
    L.append(result)

  return L

In [10]:
# summarize(tmp)

In [42]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

sentiment_model = BertForSequenceClassification.from_pretrained('Forturne/Finbert_PB',num_labels=3).to(device)
sentiment_model.eval()
sentimnet_tokenizer = BertTokenizer.from_pretrained('Forturne/Finbert_PB', truncation=True)

sentiment = pipeline("sentiment-analysis", model=sentiment_model, tokenizer=sentimnet_tokenizer, device="cuda:0")

def context_to_sentence(original_article:str):
  L = []
  L = original_article.split('. ')
  return L

def calc_sentiment(L:list)->float:
  score = 0.
  counts = 0
  for i in range(len(L)):
    if L[i]['label'] == 'Positive':
      score += L[i]['score']
      counts += 1
    elif L[i]['label'] == 'Negative':
      score -= L[i]['score']
      counts += 1

  return score/counts

def sentiment_score(original_article):
  sentence_list = context_to_sentence(original_article)
  sentiment_score = calc_sentiment(sentiment(sentence_list))
  return sentiment_score

In [43]:
def sentiment_score(article_list:list):
  total_score = 0
  for i in range(len(article_list)):
    try:
      original_article = article_list[i]
      sentence_list = context_to_sentence(original_article)
      sentiment_score = calc_sentiment(sentiment(sentence_list))
      total_score += sentiment_score
    except:
      continue
  total_score /= len(article_list)
  return total_score

In [15]:
num_list_str = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']
num_list_int = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [16]:
# origin_sector = industries[industries.index('ms_energy')]

# ARTICLE_COLLECTION = origin_sector.upper() #예시로 하나만 만들어 보면…
# article_db = db[ARTICLE_COLLECTION]
# articles = list(article_db.find({}))

In [ ]:
# articles[0]

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)
@app.route("/")
def index():
  return "<h1> Assistant of RA </h1>"

@app.route('/QA', methods=['GET'])
def QA():
    if request.method == 'GET':
        query= request.args.get('input_text')
        query_list = query.split(';')

        origin_sector = query_list[0]
        question = query_list[1]

        # sector = ms_energy
        origin_sector = industries[industries.index(origin_sector)]

        ARTICLE_COLLECTION = origin_sector.upper() #예시로 하나만 만들어 보면…
        article_db = db[ARTICLE_COLLECTION]
        articles = list(article_db.find({}))
        
        sector = articles

        tokens = question.split(' ')
        top_k = 3

        for tmp in tokens:
          if tmp in num_list_int:
            top_k = num_list_int.index(tmp) + 1
          elif tmp in num_list_str:
            top_k = num_list_str.index(tmp) + 1
        
        if ('summarize'in query_list[1]) or ('summary' in query_list[1]):
          candidates, urls = select_candidates(question, sector, num=top_k)
          summary_list = summarize(candidates)

          text = ""
          for i in range(len(summary_list)):
            text += f"Summary {i+1}: {summary_list[i]} <br>"
          text += "<br>"
          for i in range(len(candidates)):
            tmp_candidate =  context_to_sentence(candidates[i])
            if len(tmp_candidate) > 3:
              candidate = ". ".join(tmp_candidate[:3])
              url = 'aa'
              text += f"Evidence {i+1}. <br> {candidate} <br> Reference: {urls[i]} <br>"
            else:
              candidate = ". ".join(tmp_candidate)
              text += f"Evidence{ i+1}. <br> {candidate} <br> Reference: {urls[i]} <br>"

          answer = text

        elif ('sentiment' in query_list[1]) or ('trend' in query_list[1]):
          candidates, urls = select_candidates(question, sector, num=top_k)
          sentiment = sentiment_score(candidates)
          if sentiment > 0: 
            trends = 'Positive'
          else:
            trends = "Negative"

          text = f"Result: {trends}   Sentiment Score: {sentiment} <br> <br>"

          for i in range(len(candidates)):
            tmp_candidate =  context_to_sentence(candidates[i])
            if len(tmp_candidate) > 3:
              candidate = ". ".join(tmp_candidate[:3])
              text += f"Evidence {i+1}. <br> {candidate} <br> Reference: {urls[i]} <br>"
            else:
              candidate = ". ".join(tmp_candidate)
              text += f"Evidence {i+1}. <br> {candidate} <br> Reference: {urls[i]} <br>"

          answer = text

        else:
          candidates = select_candidates(question, sector, num=10)
          url = "https://d863-34-90-248-231.ngrok-free.app/QA"
          response = requests.get(url, params={'query': question, 'candidates':'@@@'.join(candidates)})

          for tmp in candidates:
            if response.text in tmp:
              candidate = tmp

          text = f"Answer: {response.text} <br> Evidence. <br> {candidate}"
          answer = text

        return answer

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e382-34-143-188-190.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Jun/2023 04:08:56] "GET / HTTP/1.1" 200 -
ERROR:__main__:Exception on /QA [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-45-c4d8fad45096>", line 78, in QA
    response = requests.get(url, params={'query': question, 'candidates':'@@@'.join(candidates)})
TypeError: sequence item 0: expected str instance, list found
INFO:werkzeug:127.0.0.1 - - [11/Jun/2023 04:09:20] "GET /QA?input

In [ ]:
origin_sector = 'ms_energy'

ARTICLE_COLLECTION = origin_sector.upper() #예시로 하나만 만들어 보면…
article_db = db[ARTICLE_COLLECTION]
articles = list(article_db.find({}))

In [ ]:
count = 0
for i in range(len(articles)):
  if "ExxonMobil is preparing" in articles[i]['content']:
    count += 1
count

In [ ]:
ms_energy[0]

In [ ]:
articles[0]